# Pose Estimation

Implementation of [detectron2](https://github.com/facebookresearch/detectron2). Most of the code was taken from [This Google Colab Notebook](https://colab.research.google.com/drive/16jcaJoc6bCFAQ96jDe2HwtXj7BMD_-m5?usp=sharing).

## Setup

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# install dependencies: (use cu101 because colab has CUDA 10.1)
!pip install -U torch==1.5 torchvision==0.6 -f https://download.pytorch.org/whl/cu101/torch_stable.html 
!pip install cython pyyaml==5.1
!pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
!gcc --version
# opencv is pre-installed on colab

In [ ]:
# install detectron2:
!pip install detectron2==0.1.3 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.5/index.html

In [ ]:
# You may need to restart your runtime prior to this, to let your installation take effect
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import cv2
import random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog

In [ ]:
#Setup for single image
single_img_path = "/content/drive/My Drive/Colab Notebooks/single.jpg"

In [ ]:
#Setup for video
video_path = '/content/drive/MyDrive/Colab Notebooks/input.mp4'
num_frames = 600
multiple_img_directory_path = "/content/drive/My Drive/Colab Notebooks/frames"
multiple_img_directory_path_outputs = "/content/drive/My Drive/Colab Notebooks/output_frames"
output_video_path = "/content/drive/My Drive/Colab Notebooks/output.mp4"

## Estimation (Single Image)

In [ ]:
if True:

  im = cv2.imread(single_img_path)
  
  cfg = get_cfg()
  cfg.merge_from_file(model_zoo.get_config_file("COCO-Keypoints/keypoint_rcnn_R_50_FPN_3x.yaml"))
  cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7
  cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Keypoints/keypoint_rcnn_R_50_FPN_3x.yaml")

  predictor = DefaultPredictor(cfg)
  outputs = predictor(im)
  outputs["instances"].pred_boxes = torch.empty((1,4))
  
  v = Visualizer(im[:,:,::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
  v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
  cv2_imshow(v.get_image()[:, :, ::-1])

## split mp4

In [ ]:
import cv2
vidcap = cv2.VideoCapture(video_path)
success,image = vidcap.read()
count = 0
while success:
  cv2.imwrite(f"{multiple_img_directory_path}/frame{count}.jpg", image)     # save frame as JPEG file      
  success,image = vidcap.read()
  print('Read a new frame: ', success)
  count += 1


## Estimation (Multiple Images)

In [ ]:
for i in range(num_frames):
  im = cv2.imread(f"{multiple_img_directory_path}/frame{i}.jpg")
  cfg = get_cfg()
  cfg.merge_from_file(model_zoo.get_config_file("COCO-Keypoints/keypoint_rcnn_R_50_FPN_3x.yaml"))
  cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7
  cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Keypoints/keypoint_rcnn_R_50_FPN_3x.yaml")

  predictor = DefaultPredictor(cfg)
  outputs = predictor(im)
  outputs["instances"].pred_boxes = torch.empty((1,4)) #replace bounding box with an empty tensor to prevent the Visualizer from drawing the bounding box

  v = Visualizer(im[:,:,::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
  v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
  # cv2_imshow(v.get_image()[:, :, ::-1])
  cv2.imwrite(f'{multiple_img_directory_path_outputs}/oframe{i}.jpg', v.get_image()[:, :, ::-1])
  print(f"frame {i} finished. {num_frames-i} to go.")

## frames to video

In [ ]:
import cv2
import glob
import re

img_array = []
numbers = re.compile(r'(\d+)')
def numericalSort(value):
  parts = numbers.split(value)
  parts[1::2] = map(int, parts[1::2])
  return parts

for filename in sorted(glob.glob(f'{multiple_img_directory_path_outputs}/*.jpg') , key=numericalSort):
  # print("read")
  img = cv2.imread(filename)
  height, width, layers = img.shape
  size = (width,height)
  img_array.append(img)

out = cv2.VideoWriter(output_video_path,cv2.VideoWriter_fourcc(*"DIVX"), 30, size)

for i in range(len(img_array)):
  out.write(img_array[i])
out.release()


print("done")

done
